In [ ]:
from gnucash_business_reports.builder import GnuCash_Data_Analysis
import numpy as np
from great_tables import GT, md
import pandas as pd
from pathlib import Path


gda = GnuCash_Data_Analysis()
gda.year = 2024

In [ ]:
gda.sanity_checker()

In [ ]:
gda.get_balance_sheet()

In [ ]:
# print(gda.get_executive_summary(include_depreciation=False))
# df = gda.get_farm_cash_transactions(include_depreciation=True)
# print(df.dtypes)
# print(df.sort_values("account_code").groupby(["account_code", "account_name"]).sum())
# print(gda.get_summary_by_account())

# flex = gda.flexible_lease_calculator()

# flex.to_csv("export/flex.csv")
# print(gda.get_corporation_value())
# print(gda.get_personal_business_expenses())

# gda.get_production().to_csv(f"export/{gda.year}-production.csv")

In [ ]:
gda.get_balance_sheet_details()

In [ ]:
grain = gda.get_grain_invoices()
open_contracts = grain[grain["Fulfilled"] == False].set_index("Invoice", drop=True)
print(open_contracts[["Crop", "Bushels", "Code"]])
open_contracts[["Crop", "Bushels", "Code"]].groupby(["Crop", "Code"]).sum()

In [ ]:
open_contracts[["Crop", "Bushels", "Code"]].groupby(["Crop", "Code"]).sum().to_html()

In [ ]:
# gda.get_all_cash_transactions().groupby(["account_desc"]).agg("post_date", np.max)
# latest_tx = gda.get_all_transactions().groupby(["account_desc"]).agg(max_date=("post_date", max))
latest_tx = gda.get_all_transactions().groupby(["account_desc"])["post_date"].max() #.reset_index().set_index("account_desc")
latest_tx

In [ ]:
latest = latest_tx.loc[latest_tx.index.str.match("Delivered")]
last_delivery = latest.max()

In [ ]:
gda.year = 2024
grain = (
            gda.get_commodity_stock_values(["account_name", "account_desc", "commodity_guid"])
        ).reset_index().set_index(["account_desc"])
grain["abs_qty"] = abs(grain["qty"])
grain = grain.join(latest_tx)
grain[["abs_qty", "post_date"]]

In [ ]:
df = pd.pivot_table(grain, values="abs_qty", index="account_name", columns="account_desc").fillna(0)
df["Contracted"] = df["Contracted Corn"] + df["Contracted Soybeans"]
df["Delivered"] = df["Delivered Corn"] + df["Delivered Soybeans"]
df["Harvested"] = df["Harvested Corn"] + df["Harvested Soybeans"]
df = df[["Contracted", "Delivered", "Harvested"]]
df["Total"] = df["Delivered"] + df["Harvested"]
df["raw_pct"] = df["Delivered"] / df["Contracted"]
df = df.reset_index()

In [ ]:
def create_bar(prop_fill: float, max_width: int, height: int) -> str:
    """Create divs to represent prop_fill as a bar."""
    if prop_fill > 1:
        prop_fill = 1
    width = round(max_width * prop_fill, 2)
    px_width = f"{width}px"
    return f"""\
    <div style="width: {max_width}px; background-color: lightgrey;">\
        <div style="height:{height}px;width:{px_width};background-color:green;"></div>\
    </div>\
    """
zoom_level = 100

In [ ]:
df["icon"] = df["account_name"].str.lower() + ".png"

# df["guid"].apply(lambda x: find_parent(x)).str.rstrip(">")

df["Progress"] = df["raw_pct"].apply(lambda x: create_bar(x, max_width=75*(zoom_level/100), height=20*(zoom_level/100)))

res = df[["icon", "Contracted", "Delivered", "Harvested", "Total", "Progress"]]
res

In [ ]:
table = (
    GT(res, rowname_col="icon")
    .tab_header(title=f"{gda.year} Harvest",
                subtitle="Progress towards filling contracts"
                )
    .tab_stubhead(label="Crop")
    # .tab_spanner("Earnings", cs.contains("Earnings"))
    .fmt_number(["Contracted", "Delivered", "Harvested", "Total"], decimals=0)    
    # .cols_label(**{
    #     "Total Earnings": "Total $M",
    #     "Off-the-Field Earnings": "Off field $M",
    #     "Off-the-Field Earnings Perc": "Off field %"
    # })
    # .fmt_number(["Total Earnings", "Off-the-Field Earnings"], scale_by = 1/1_000_000, decimals=1)
    .fmt_image("icon", path="./img/")
    .tab_source_note(
        md(
            '<br><div style="text-align: center;">'
            "GNUCash / Meadowland Portal"
            f" | Last Recorded Delivery: {last_delivery.strftime('%Y-%m-%d')}"
            "</div>"
            "<br>"
        )
    )
)
table

In [ ]:
path = Path(f"{gda.get_config()["Paths"]["html"]}/contract-progress.html")
path.write_text(table.as_raw_html())

In [ ]:
gnuc_prod = gda.get_production().reset_index() \
            .rename(columns={"operation": "Farm",
                             "crop": "Crop",
                             }) \
            .drop(columns=["bu_per_acre", "operation_id"]) \
            .groupby(["Farm", "Crop"]) \
            .sum(numeric_only=True)
gnuc_prod

In [ ]:
def filter_rows_by_values(df, col, values):
    return df[~df[col].isin(values)]

jd_data = pd.read_excel("/home/justin/Downloads/Harvest_2024 (1).xlsx")
jd_data = filter_rows_by_values(jd_data, "Varieties", ["---"])
jd_data = filter_rows_by_values(jd_data, "Moisture", ["---"])
jd_data = filter_rows_by_values(jd_data, "Farms", ["Morin Currie Farms"])
jd_data = jd_data.rename(columns={"Farms": "Farm",
                             "Crop Type": "Crop",
                             }) \
    .groupby(["Farm", "Crop"]) \
    .agg(Harvested=("Area Harvested", "sum"),\
         Dry_Yield=("Total Dry Yield", "sum"), \
         Moisture=("Moisture", "mean"), \
         Date=("Last Harvested", "max")) \
    .reset_index() \
    .set_index(["Farm", "Crop"])
jd_data.head()

In [ ]:
production = jd_data.join(gnuc_prod).fillna(0)
production["New Bushels"] = production["Dry_Yield"] - production["total_bushels"]
production.head(10)

In [ ]:
production["Bu/Acres Worked"] = production["total_bushels"] / production["Harvested"]
production["Bu/Total Acres"] = production["total_bushels"] / production["acres"]
production = production.rename(columns={"operation": "Farm",
                                        "acres": "Total Acres",
                                        "total_bushels": "Bushels",
                                        "Harvested": "Acres Worked"}) \
                                .sort_values("Date")
production

In [ ]:
prod_res = production.reset_index()
prod_res["icon"] = prod_res["Crop"].str.lower() + ".png"
prod_res = prod_res[["icon", "Farm", "Bushels", "Acres Worked", "Total Acres", "Bu/Acres Worked", "Bu/Total Acres", "Moisture"]]
prod_res

In [ ]:
prod_gt = GT(prod_res, rowname_col="icon").tab_header(title=f"{gda.year} Production YTD",
            subtitle="Based on John Deere Operations Center Data") \
               .tab_stubhead(label="Crop") \
               .fmt_number(["Bushels", "Total Acres", "Acres Worked", "Bu/Acres Worked", "Bu/Total Acres"], decimals=0) \
               .fmt_number(["Bu/Acres Worked", "Bu/Total Acres", "Moisture"], decimals=1) \
               .fmt_image("icon", path="./img/")
prod_gt

In [ ]:
path = Path(f"{gda.get_config()["Paths"]["html"]}/production.html")
path.write_text(prod_gt.as_raw_html())

In [ ]:
prod_gt.as_raw_html()

In [ ]:
from jinja2 import Environment, FileSystemLoader
from gnucash_business_reports.config import get_config

file_loader = FileSystemLoader("templates")
env = Environment(loader=file_loader) # defaults are fine
guts = prod_gt.as_raw_html()
template = env.get_template("printable.html")

report_details = {
        "report_name": f"{gda.year} Transaction Detail Report",
        "organization_name": f"""{get_config()["Organization"]["business_name"]}""",
    }
output = template.render(
    guts=guts,
    details=report_details
)
with open(f"templates/{gda.year}-Detail_Report.html", "w") as f:
    f.write(output)

In [ ]:
def iconize(column):
    return column.str.lower() + ".png"

In [ ]:
def trend_data(starting_year, years_to_go_back, index_col, trend_col):
    trend_gda = GnuCash_Data_Analysis()
    for x in range(years_to_go_back):
        trend_gda.year = starting_year - years_to_go_back + x + 1
        df = trend_gda.get_executive_summary(False)
        df["year"] = trend_gda.year
        if x == 0:
            trend_df = df.set_index(["year", index_col])
        else:
            trend_df = pd.concat([trend_df, df.set_index(["year", index_col])])
    del(trend_gda)
    return pd.pivot_table(trend_df, values=trend_col, index=index_col, columns="year").fillna(0)

trends = trend_data(2024, 7, "Account", "Amount")
trends

In [ ]:
# trend_data = [[1, 2, 5],[1, 5, 3],[3, 2, 1],[1, 3, 5],[5, 1, 3]]
trend_data = [{ "val": trends.loc["INCOME"].tolist() },
            { "val": trends.loc["EXPENSE"].tolist() },
            { "val": trends.loc["OIBDA"].tolist() },
            { "val": trends.loc["ASSET"].tolist() },
            { "val": trends.loc["LIABILITY"].tolist() }]
len(trend_data)

In [ ]:
exec_summ = gda.get_executive_summary(include_depreciation=False)
exec_summ["icon"] = iconize(exec_summ["Account"])
exec_summ["Trend"] = trend_data
exec_summ.dtypes

In [ ]:
GT(exec_summ, rowname_col="icon").tab_header(title=f"{gda.year} Executive Summary",
            subtitle="Overview") \
               .fmt_number(["Amount"], decimals=2, pattern="${x}",) \
               .fmt_image("icon", path="./img/") \
               .fmt_nanoplot(columns="Trend", plot_type="bar")